In [1]:
import importlib
from model.models.model_label_embedding import ModelLabelEmbedding
from model.models.model_converter import ModelConverter
import torch

## Модель явного конвертера

Предобученная модель явного конвертера работает с двумя корпусами: СинТагРус (syntagrus) и НКРЯ (snyatnik).
Загрузим эту модель. Здесь индекс СинТагРуса — 0, а индекс НКРЯ — 1.

In [ ]:
config_converter = importlib.import_module('configs.model_converter.config').config
config_converter['datasets_names'] = ['syntagrus', 'snyatnik']
model_converter = ModelConverter.load_from_checkpoint(
    r'checkpoints\model_converter\syntagrus+snyatnik.ckpt', config=config_converter, map_location=torch.device('cpu'))

Получить предсказания для одного предложения можно с помощью метода `predict`. На вход методу подается список слов предложения, морфологическая разметка в стандарте СинТагРуса или НКРЯ (формат помет такой же, который используется в исходных файлах этих корпусов), индекс морфологического стандарта на входе и индекс морфологического стандарта на выходе. Ниже осуществляется конвертация разметки предложения "Защитит его бог!" из морфологического стандарта НКРЯ в морфологический стандарт СинТагРуса и показывается, что при смене порядка слов пропадает ошибка выбора значения категории лица для словоформы "защитит".

In [3]:
model_converter.predict(['Защитит', 'его', 'бог', '!'], 
                        ['V sg pf tran act indic fut 3p', 'SPRO m sg acc 3p', 'S m anim sg nom', 'PUNCT'], 
                        1, 0)

['V СОВ ИЗЪЯВ НЕПРОШ ЕД 2-Л', 'S ЕД МУЖ ВИН ОД', 'S ЕД МУЖ ИМ ОД', 'PUNCT']

In [4]:
model_converter.predict(['Бог', 'его', 'защитит', '!'], 
                        ['S m anim sg nom', 'SPRO m sg acc 3p', 'V sg pf tran act indic fut 3p', 'PUNCT'], 
                        1, 0)

['S ЕД МУЖ ИМ ОД', 'S ЕД МУЖ ВИН ОД', 'V СОВ ИЗЪЯВ НЕПРОШ ЕД 3-Л', 'PUNCT']

Посмотрим на конвертацию в обратную сторону:

In [5]:
model_converter.predict(['Защитит', 'его', 'бог', '!'], 
                        ['V СОВ ИЗЪЯВ НЕПРОШ ЕД 2-Л', 'S ЕД МУЖ ВИН ОД', 'S ЕД МУЖ ИМ ОД', 'PUNCT'], 
                        0, 1)

['V sg pf tran act indic fut 2p',
 'SPRO m sg acc 3p',
 'S m anim sg nom',
 'PUNCT']

In [6]:
model_converter.predict(['Бог', 'его', 'защитит', '!'], 
                        ['S ЕД МУЖ ИМ ОД', 'S ЕД МУЖ ВИН ОД', 'V СОВ ИЗЪЯВ НЕПРОШ ЕД 2-Л', 'PUNCT'], 
                        0, 1)

['S m anim sg nom',
 'SPRO m sg acc 3p',
 'V sg pf tran act indic fut 3p',
 'PUNCT']

## Модель неявного конвертера

Предобученная модель неявного конвертера фактически представляет из себя модель автоматической морфологической разметки, которая осуществляет разметку в любом из морфологических стандартов корпусов, предложенных организаторами соревнования MorphoRuEval-2017. Загрузим эту модель. Здесь индекс корпуса SynTagRus — 0, RNC — 1, GICR — 2, OpenCorpora — 3.

In [ ]:
config_lel = importlib.import_module('configs.model_label_embedding.config').config
config_lel['datasets_names'] = ['syntagrus_dialogue', 'rnc_dialogue', 'gikrya_dialogue', 'opencorpora_dialogue']
model_lel = ModelLabelEmbedding.load_from_checkpoint(
    r'checkpoints\model_label_embedding\morphorueval.ckpt', config=config_lel, map_location=torch.device('cpu'))

Получить предсказания для одного предложения также можно с помощью метода `predict`. На вход подается только список слов предложения и индекс корпуса, в морфологическом стандарте которого хочется получить разметку.

In [8]:
model_lel.predict(['К', 'Lexus', 'Спилберг', 'неравнодушен', 'ещё', 'и', 'потому', ',', 'что', 'сам', 'ездит', 'на', 'RX', '300', '.'], 0)

['ADP',
 'NOUN Animacy=Inan Case=Dat Gender=Neut Number=Sing',
 'NOUN Animacy=Anim Case=Nom Gender=Masc Number=Sing',
 'ADJ Degree=Pos Gender=Masc Number=Sing Variant=Short',
 'ADV Degree=Pos',
 'PART',
 'ADV Degree=Pos',
 'PUNCT',
 'CONJ',
 'ADJ Case=Nom Degree=Pos Gender=Masc Number=Sing',
 'VERB Mood=Ind Number=Sing Person=3 Tense=Notpast VerbForm=Fin Voice=Act',
 'ADP',
 'NOUN Animacy=Inan Case=Loc Gender=Masc Number=Sing',
 'NUM NumForm=Digit',
 'PUNCT']

In [9]:
model_lel.predict(['К', 'Lexus', 'Спилберг', 'неравнодушен', 'ещё', 'и', 'потому', ',', 'что', 'сам', 'ездит', 'на', 'RX', '300', '.'], 1)

['ADP',
 'X',
 'NOUN Animacy=Anim Case=Nom Gender=Masc Number=Sing',
 'ADJ Gender=Masc Number=Sing Variant=Short',
 'ADV Degree=Pos',
 'CONJ',
 'ADV Degree=Pos',
 'PUNCT',
 'CONJ',
 'DET Case=Nom Gender=Masc Number=Sing',
 'VERB Mood=Ind Number=Sing Person=3 Tense=Notpast VerbForm=Fin Voice=Act',
 'ADP',
 'X',
 'NUM NumForm=Digit',
 'PUNCT']

In [10]:
model_lel.predict(['К', 'Lexus', 'Спилберг', 'неравнодушен', 'ещё', 'и', 'потому', ',', 'что', 'сам', 'ездит', 'на', 'RX', '300', '.'], 2)

['ADP',
 'NOUN Animacy=Inan Case=Dat Gender=Masc Number=Sing',
 'NOUN Animacy=Anim Case=Nom Gender=Masc Number=Sing',
 'ADJ Degree=Pos Gender=Masc Number=Sing Variant=Short',
 'ADV Degree=Pos',
 'PART',
 'ADV Degree=Pos',
 'PUNCT',
 'CONJ',
 'ADJ Case=Nom Degree=Pos Gender=Masc Number=Sing',
 'VERB Mood=Ind Number=Sing Person=3 Tense=Notpast VerbForm=Fin Voice=Act',
 'ADP',
 'NUM NumForm=Digit',
 'NUM NumForm=Digit',
 'PUNCT']

In [11]:
model_lel.predict(['К', 'Lexus', 'Спилберг', 'неравнодушен', 'ещё', 'и', 'потому', ',', 'что', 'сам', 'ездит', 'на', 'RX', '300', '.'], 3)

['ADP',
 'X',
 'PROPN Animacy=Anim Case=Nom Gender=Masc Number=Sing',
 'ADJ Case=Nom Gender=Masc Number=Sing Variant=Short',
 'ADV',
 'CONJ',
 'ADV',
 'PUNCT',
 'CONJ',
 'DET Case=Nom Gender=Masc Number=Sing',
 'VERB Aspect=Imp Mood=Ind Number=Sing Person=3 Tense=Notpast VerbForm=Fin',
 'ADP',
 'X',
 'NUM Form=Digit',
 'PUNCT']